<a href="https://colab.research.google.com/github/mtedder/AI-ML-Workshop/blob/master/notebooks/MPG_Regression_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MPG Regression - Predict Fuel Efficiency Keras Project

[Ref Docs 1](https://www.tensorflow.org/tutorials/keras/basic_regression)



##Imports

In [0]:
#Regression example - https://www.tensorflow.org/tutorials/keras/basic_regression
#https://towardsdatascience.com/getting-data-into-tensorflow-estimator-models-3432f404a8da
#Ref: https://www.tensorflow.org/api_docs/python/tf
#Data - How to get it
import tensorflow as tf
sess = tf.Session()
from keras import backend as K
K.set_session(sess)
K.set_learning_phase(0) # all new operations will be in test mode from now on

import os
#import tensorflow as tf
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from tensorflow import keras

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl, utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD

#imports for online predictions
import googleapiclient 
from googleapiclient import discovery
from googleapiclient import errors


from google.colab import auth
from google.colab import files

#from tensorflow.keras import layers
# Import the `pyplot` module
import matplotlib.pyplot as plt

##Version Info

In [0]:
#!python --version
# print("Tensorflow version" + tf.__version__)
##VIEW DATASET
# dataset_path = tf.keras.utils.get_file("auto-mpg.data", "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
# !cat dataset_path
# %cd datasets/
# !ls -al
# !cat auto-mpg.data

##Data Preparation

In [0]:
#MPG regression data
#Downloads a file from a URL to /root/.keras/datasets/auto-mpg.data if it not already in the cache. Store path to file in dataset_path variable - https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file
dataset_path = tf.keras.utils.get_file("auto-mpg.data", "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
#Create Column names list because file data does not have a header row
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 'Acceleration', 'Model_Year', 'Origin']
#Read downloaded csv file and convert to dataframe - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html#pandas.read_csv
raw_dataset = pd.read_csv(dataset_path, names=column_names, na_values = "?", comment='\t', sep=" ", skipinitialspace=True)

#Make a copy of this object’s indices and data - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.copy.html#pandas.DataFrame.copy
dataset = raw_dataset.copy()

#Return the last n row(s) default n = 5 - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.tail.html#pandas.DataFrame.tail
#dataset.tail()
dataset.head(40) #NaN value on line 32 col. 4

#Detect missing values & Return the sum of the values for the requested axis - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Panel.isna.html#pandas.Panel.isna
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sum.html#pandas.DataFrame.sum
dataset.isna().sum()

#Remove missing values NaN etc...- http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna
dataset = dataset.dropna()

#Convert categorical Origin column to one-hot - https://developers.google.com/machine-learning/glossary/#one-hot_encoding
#https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/: "In fact, using this encoding and allowing the model to assume a natural ordering between categories may result in poor performance or unexpected results (predictions halfway between categories)."
#Return item and drop from frame - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pop.html#pandas.DataFrame.pop
origin = dataset.pop('Origin')

dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0

In [0]:
##Workspace cell
#dataset.head(40)
#dataset.size
# dataset = dataset.dropna()
# dataset.isna().sum()

##Feature Extraction

In [0]:
#Now split the data into a train and a test set
#Return a random sample of items from an axis of object - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html#pandas.DataFrame.sample
train_dataset = dataset.sample(frac=0.8,random_state=0)
#Drop specified labels from rows or columns with the given indecies (index)- http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html#pandas.DataFrame.drop
test_dataset = dataset.drop(train_dataset.index)

#Inspect the data using matplotlib - https://matplotlib.org/
#http://pandas.pydata.org/pandas-docs/stable/visualization.html
#https://matplotlib.org/api/mlab_api.html?highlight=kde#matplotlib.mlab.GaussianKDE
# l = plt.mlab.GaussianKDE(train_dataset["MPG"])
# plt.show()

#Or use dataframe plot function to get Kdensity plot - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html
#https://towardsdatascience.com/histograms-and-density-plots-in-python-f6bda88f5ac0
#train_dataset["MPG"].plot(use_index=False, kind="kde")


#Look at the overall statistics
#Generates descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html#pandas.DataFrame.describe
train_stats = train_dataset.describe()
# print(train_stats)

#Return item and drop from frame - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pop.html#pandas.DataFrame.pop
train_stats.pop("MPG")

#Transpose index and columns (for display purposes - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.transpose.html#pandas.DataFrame.transpose
train_stats = train_stats.transpose()

print(train_stats)

#get labels from the features train dataset - http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pop.html#pandas.DataFrame.pop
train_labels = train_dataset.pop('MPG')

#get labels from the features test dataset
test_labels = test_dataset.pop('MPG')

#normalize the train and test data to bring values to the same scale using standard score calculation - https://en.wikipedia.org/wiki/Standard_score
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

#Pandas Dataframe
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

normed_train_data.head()

##Build Model
Ref:

In [0]:

#MPG data model
EPOCHS = 100

# model = keras.models.Sequential()
# model.add(Dense(64, activation=tf.nn.relu, input_shape=(len(train_dataset.keys()),), name='feature'))
# model.add(Dense(64, activation=tf.nn.relu, name='midLayer'))
# model.add(Dense(1, name='outLayer'))

# model = keras.models.Sequential()
# model.add(keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(len(normed_train_data.keys()),), name='feature'))
# model.add(keras.layers.Dense(64, activation=tf.nn.relu, name='midLayer'))
# model.add(keras.layers.Dense(1, name='outLayer'))

model = Sequential()
model.add(Dense(64, activation=tf.nn.relu, input_dim=(len(normed_train_data.keys()))))
# model.add(Activation('tanh'))
model.add(Dense(64, activation=tf.nn.relu, name='midLayer'))
model.add(Dense(1))

# model = keras.models.Sequential()
# model.add(keras.layers.Dense(64, activation=tf.nn.relu, input_dim=(len(normed_train_data.keys()))))
# model.add(keras.layers.Dense(64, activation=tf.nn.relu, name='midLayer'))
# model.add(keras.layers.Dense(1))


# model.add(Activation('sigmoid'))
# sgd = SGD(lr=0.1)
#Define an Optimizer that implements the RMSProp algorithm with a learning rate (step size) of .001 - https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer
#http://www.cs.toronto.edu/%7Etijmen/csc321/slides/lecture_slides_lec6.pdf
#Specifies algorithm to help with convergence to the minimum error
#https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a
#https://engmrk.com/rmsprop/?utm_campaign=News&utm_medium=Community&utm_source=DataCamp.com
optimizer = tf.train.RMSPropOptimizer(0.001)

#Compile Configures the model for training - https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile
#Use 'mae' and 'mse' list of metrics to be evaluated by the model during training and testing by calculating these they can be used to track model accuracy
#Use 'mse' to compute loss - https://developers.google.com/machine-learning/crash-course/descending-into-ml/training-and-loss
# model.compile(loss='mse',
#                 optimizer=optimizer,
#                 metrics=['mae', 'mse', 'accuracy'])
model.compile(loss='mse',
                optimizer=optimizer, metrics=['mae', 'mse']) #We'll use the mean square error, a standard loss for regression problems. https://www.tensorflow.org/guide/low_level_intro

#print a simple description of the model - https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#summary
model.summary()

##Train Model
Ref:

In [0]:
#Not needed for Cloud ML Serving
#Train the model
#Test the untrained model using a small batch of 10 samples to test for errors
#by calling the predict function which generates output predictions for the input samples - https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#predict
#example_batch = normed_train_data[:10]
#example_result = model.predict(example_batch)
#example_result
#If it runs without errors, start training the model
EPOCHS = 1000

# train_input_fn = tf.estimator.inputs.pandas_input_fn(
#     x=normed_train_data,
#     y=train_labels,
#     batch_size=100,
#     num_epochs=EPOCHS,
#     shuffle=True,
#     queue_capacity=1000,
#     num_threads=1
# )


#tf.keras.backend.clear_session() #https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session

#Train using fit which Trains the model for a fixed number of epochs (iterations on a dataset) - https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit
#validation_split of .2. Fraction of the training data to be used as validation data
history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0)

#Alternate training using EarlyStopping callback - https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
# The patience parameter is the amount of epochs to check for improvement.
# Early stopping is a useful technique to prevent overfitting.
# early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# history = model.fit(
#     normed_train_data, train_labels,
#     epochs=EPOCHS,
#     validation_split = 0.2, verbose=0, callbacks=[early_stop])

#Visualize the model's training progress
#History.history attribute is a record of training loss values and metrics values at successive epochs, as well as validation loss values and validation metrics values (if applicable)
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


##Evaluate Model

In [0]:
#Not needed for Cloud ML Serving
#Is this model good? Visualize model performance
plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error [MPG]')

plt.plot(hist['epoch'], hist['mean_absolute_error'], label='Train Error')
  
plt.plot(hist['epoch'], hist['val_mean_absolute_error'], label = 'Val Error')
plt.legend()
plt.ylim([0,5])

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Mean Square Error [$MPG^2$]')
plt.plot(hist['epoch'], hist['mean_squared_error'],
         label='Train Error')
plt.plot(hist['epoch'], hist['val_mean_squared_error'],
         label = 'Val Error')
plt.legend()
plt.ylim([0,20])

#Test model using the test data and evaluate
#Returns the loss value & metrics values for the model in test mode - https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#evaluate
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))


##Inference – Make Predictions

In [0]:
#Not needed for Cloud ML Serving
#Use test data to make predictions on trained model
#Predict Generates output predictions for the input sample - https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#predict
#Then flatten numpy array result - https://www.tutorialspoint.com/numpy/numpy_ndarray_flatten.htm
results_predictions = model.predict(normed_test_data)
test_predictions = results_predictions.flatten()

#Plot the results
plt.figure()
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

#Plot error Histogram
plt.figure()
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")


#Deploy to ML Cloud Engine
[Tensorflow serving Ref](https://www.tensorflow.org/tfx/guide/serving)

[Preparing Models for the cloud](https://towardsdatascience.com/freezing-a-keras-model-c2e26cb84a38)

In [0]:
# !rm -rf 1

##Prepare Model for Saving

In [0]:
# Constants
model_version = "1" #Change this to export different model versions, i.e. 2, ..., 7

#import tensorflow as tf
#setting values for the sake of saving the model in the proper format
#http://amygdala.github.io/ml/tensorflow/cloud_ml_engine/2018/01/26/tf.html
x = model.input
y = model.output

# receiver_tensors = {
#     'Cylinders' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'Displacement' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'Horsepower' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'Weight' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'Acceleration' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'Model_Year' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'USA' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'Europe' : tf.placeholder(shape=[None,1], dtype=tf.float32),
#     'Japan' : tf.placeholder(shape=[None,1], dtype=tf.float32)
#   }

# receiver_tensors = {
#     'Cylinders' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'Displacement' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'Horsepower' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'Weight' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'Acceleration' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'Model_Year' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'USA' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'Europe' : tf.placeholder(shape=[1,1], dtype=tf.float32),
#     'Japan' : tf.placeholder(shape=[1,1], dtype=tf.float32)
#   }

#Start tried this!
# https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/census/tensorflowcore/trainer/task.ipynb
# https://github.com/GoogleCloudPlatform/cloudml-samples/issues/67
# INPUT_COLUMNS = normed_train_data.columns
# def input_fn():
#   """Build the serving inputs."""
#   inputs = {}
#   for feat in INPUT_COLUMNS:
#     inputs[feat] = tf.placeholder(shape=[None,1], dtype=tf.float32)
  
#   return inputs

# inputs_dict = input_fn()

# inputs_info = {
#         name: tf.saved_model.utils.build_tensor_info(tensor)
#         for name, tensor in inputs_dict.items()
#     }

# signature_def = tf.saved_model.signature_def_utils.build_signature_def(
#       inputs=inputs_info,
#       outputs={"prediction":tf.saved_model.utils.build_tensor_info(tf.placeholder(shape=[None,1], dtype=tf.float32))},
#       method_name=signature_constants.PREDICT_METHOD_NAME
#   )
#End tried this!

# signature_def = tf.saved_model.signature_def_utils.regression_signature_def(
#       inputs_info,
#       {"prediction":tf.saved_model.utils.build_tensor_info(tf.placeholder(shape=[None,1], dtype=tf.float32))}
# #       method_name=signature_constants.PREDICT_METHOD_NAME
#   )

# print ('Results of Model', model.predict_proba(normed_train_data))
# print ('Results of Model', model.predict_proba(x_df))
# tensor_info_x = utils.build_tensor_info(x)
# tensor_info_y = utils.build_tensor_info(y)

#prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def(receiver_tensors, {"prediction":y}) #for MPG model
prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def({"inputs":x}, {"prediction":y}) #for MPG model - use this one

# valid_prediction_signature = tf.saved_model.signature_def_utils.is_valid_signature(prediction_signature)
# if(valid_prediction_signature == False):
#     raise ValueError("Error: Prediction signature not valid!")

builder = saved_model_builder.SavedModelBuilder('./'+ model_version)
#legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
#legacy_init_op = tf.group(tf.global_variables_initializer(), name='legacy_init_op')
legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

# ex.1
# init_local = tf.global_variables_initializer() #tf.local_variables_initializer()
# init_tables = tf.tables_initializer()
# legacy_init_op = tf.group(init_local, init_tables)

# Add the meta_graph and the variables to the builder
builder.add_meta_graph_and_variables(
      sess, [tag_constants.SERVING],
      signature_def_map={
           signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:prediction_signature,
#          signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:signature_def, #ex.1
      },
      #main_op=tf.tables_initializer())
      legacy_init_op=legacy_init_op)

# save the graph
builder.save()

##Save trained model to OS drive (optional not required for serving)

In [0]:
##Export/Save trained model
#https://cloud.google.com/ml-engine/docs/scikit/deploying-models
#https://www.tensorflow.org/guide/saved_model#performing_the_export
#https://www.tensorflow.org/tutorials/keras/save_and_restore_models
#https://medium.com/tensorflow/training-and-serving-ml-models-with-tf-keras-fd975cc0fa27
#https://www.tensorflow.org/api_docs/python/tf/saved_model/simple_save
#https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_saved_model.py#L100
#https://towardsdatascience.com/deploy-tensorflow-models-9813b5a705d5

#graph1 = tf.Graph()
tf.keras.backend.set_learning_phase(0) # Ignore dropout at inference
tf.keras.backend.clear_session() #https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session

cwd = os.getcwd()
export_path = os.path.join(cwd, 'testmodel1')

#https://towardsdatascience.com/deploy-tensorflow-models-9813b5a705d5
#tensor_info_output = tf.saved_model.utils.build_tensor_info(model.outputs[0])
out = tf.placeholder(shape=model.outputs[0].get_shape(), dtype=model.outputs[0].dtype, name='myresult')
#out = tf.estimator.export.ClassificationOutput(classes=tf.placeholder(tf.strings, name='outresult'))

#Keras save model method #2 - using the simple_save function 
with tf.keras.backend.get_session() as sess:
  tf.saved_model.simple_save(
      sess,
      export_path,      
      inputs= json_serving_input_fn().features,
      outputs={'result':out})  


In [0]:
#https://colab.research.google.com/notebooks/io.ipynb#scrollTo=hauvGV4hV-Mh
from google.colab import files
files.download('testmodel1/saved_model.pb')

##Upload model to existing GCS bucket

In [0]:
#https://colab.research.google.com/notebooks/io.ipynb#scrollTo=xM70QWdxeE7q
auth.authenticate_user()

# Existing bucket name
# (GCS buckets are part of a single global namespace.)
bucket_name = '#INSERT YOUR BUCKET NAME HERE!!'

# Copy the model directory to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp -r INSERT MODEL DIRECTORY NAME HERE/. gs://{bucket_name}/


##Upload GOOGLE_APPLICATION_CREDENTIALS json file

In [0]:
#Upload GOOGLE_APPLICATION_CREDENTIALS json file from local computer and save to this notebook
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

##Create GOOGLE_APPLICATION_CREDENTIALS environment variable

In [0]:
#set GOOGLE_APPLICATION_CREDENTIALS environment variable
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'INSERT YOUR CREDENTIALS FILENAME HERE!!'
version=None

##Request online prediction from deployed model

In [0]:
#Setup online cloud model
#https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#requesting_predictions

#Setup online cloud model
#https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#requesting_predictions
#https://cloud.google.com/ml-engine/docs/v1/predict-request
# https://cloud.google.com/ml-engine/docs/tensorflow/python-client-library
#https://www.raspberrypi.org/magpi/tensorflow-ai-raspberry-pi/
#https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.98614&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false
#https://stackoverflow.com/questions/45705070/how-to-load-and-use-a-saved-model-on-tensorflow
Project_ID = 'mltest-229502'
MODEL_NAME = 'MpgModel'
MODEL_VERSION_NAME = 'MpgModel2'

# Create the ML Engine service object.
# To authenticate set the environment variable
# GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
#name = 'projects/{}/models/{}'.format(Project ID, 'MpgModel')
service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(Project_ID,  MODEL_NAME)

#if version is not None:
name += '/versions/{}'.format(MODEL_VERSION_NAME)

##Test data taken from first row (index 9) of normed_test_data
response = service.projects().predict(
    name=name,
    body={"instances": [{"inputs": [1.483887,1.865988,2.234620,1.018782,-2.530891,-1.604642,0.774676,-0.465148,-0.495225]}]} #this WORKS! with the mpg model
).execute()

# if 'error' in response:
#     raise RuntimeError(response['error'])

#expect [16.888962] for inputs Cylinders,Displacement,Horsepower,Weight,Acceleration,Model_Year,USA,Europe,Japan
#                              [1.483887,1.865988,2.234620,1.018782,-2.530891,-1.604642,0.774676,-0.465148,-0.495225]
  
response

##Loading and running a prediction from a saved file

In [0]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tag_constants.SERVING], "1")
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("dense_1_input:0") #input layer name maps to tensor in the graph
    model = graph.get_tensor_by_name("dense_2/BiasAdd:0") #output layer name   
    print(sess.run(model, feed_dict={"dense_1_input:0": [[1.483887,1.865988,2.234620,1.018782,-2.530891,-1.604642,0.774676,-0.465148,-0.495225]]}))